In [2]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/bin/python3.12'

In [3]:
import pandas as pd

pd.set_option("display.max_rows", 102)
from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader, TensorDataset

from src.utils import (
    load_MovieLens,
    create_test_user,
    train_test_val_split,
    seed_everything,
    UserMovieDataset,
    split_test_df,
    add_not_watched_movies,
    create_test_user_display_df,
    train,
    predict,
)
from src.models import NCF
from src.metrics import reccomendation_report

## Constants

In [4]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7

In [5]:
seed_everything(RANDOM_STATE)

## Data

In [6]:
df_users, df_movies, df_ratings = load_MovieLens(DATA_FOLDER)

### Creating a test_user

In [7]:
df_users, df_ratings, new_user_id = create_test_user(
    df_users, df_ratings, [6, 16, 3192, 1461, 827, 887, 593]
)
print("Test user watch list:")
display(df_ratings[df_ratings.userId == new_user_id].merge(df_movies, on="movieId"))

Test user watch list:


,userId,movieId,rating,timestamp,name,genre
0,6040,6,5,0,Sabrina (1995),Comedy|Romance
1,6040,16,5,1,Sense and Sensibility (1995),Drama|Romance
2,6040,3192,5,2,Singles (1992),Comedy|Drama|Romance
3,6040,1461,5,3,Love and Other Catastrophes (1996),Romance
4,6040,827,5,4,Emma (1996),Comedy|Drama|Romance
5,6040,887,5,5,Singin' in the Rain (1952),Musical|Romance
6,6040,593,5,6,Pretty Woman (1990),Comedy|Romance


### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 20% of test.

In [8]:
df_train, df_test, df_val = train_test_val_split(df_ratings, df_movies, RANDOM_STATE)

Enriching test:   0%|          | 0/4832 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/1209 [00:00<?, ?it/s]

### Loading additional data

In [9]:
# cos_dist = computeCosineSimilarities(df_train, "userId", "movieId", df_movies["movieId"].nunique())

# with open("../../data/cos_dist.pt", "wb") as f:
#     torch.save(cos_dist, f)

with open("../../data/cos_dist.pt", "rb") as f:
    cos_dist = torch.load(f)
popularity = torch.bincount(
    torch.tensor(df_train.movieId.values), minlength=df_movies["movieId"].nunique()
)
popularity = popularity / torch.max(popularity)

## NCF Best

In [10]:
seed_everything(RANDOM_STATE)
train_loader = DataLoader(
    UserMovieDataset(df_train, df_movies["movieId"].nunique()),
    batch_size=2048,
    shuffle=True,
)
val_loader = DataLoader(
    TensorDataset(torch.tensor(df_val["userId"]), torch.tensor(df_val["movieId"])),
    batch_size=4096,
    shuffle=False,
)
test_loader = DataLoader(
    TensorDataset(torch.tensor(df_test["userId"]), torch.tensor(df_test["movieId"])),
    batch_size=4096,
    shuffle=False,
)
num_users = df_users["userId"].nunique()
num_items = df_movies["movieId"].nunique()

  0%|          | 0/994175 [00:00<?, ?it/s]

**Best hyper parametres combination after tuning:**

Trial 1 finished with value: 0.6512141227722168 and parameters: {

'n_mlp_layers': 6, 

'mlp_layers_dim': 496.0, 

'mlp_kwargs_dropout': False, 

'mlp_kwargs_batchnorm': False, 

'mlp_kwargs_dropout_rate': 0.5587254241057561, 

'lr': 0.004005098808222355, 

'n_epochs': 7.0

}

In [11]:
seed_everything(RANDOM_STATE)
ncf = NCF(
    num_users,
    num_items,
    n_mlp_layers=6,
    mlp_layers_dim=496,
    mlp_kwargs={
        "activation": True,
        "dropout": False,
        "batchnorm": False,
        "dropout_rate": 0.56,
    },
)
display(ncf)

NCF(
  (user_embedding): Embedding(6041, 248)
  (item_embedding): Embedding(3883, 248)
  (mlp): Sequential(
    (MLP_layer_0): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=496, out_features=496, bias=True)
        (Activation): ReLU()
      )
    )
    (MLP_layer_1): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=496, out_features=496, bias=True)
        (Activation): ReLU()
      )
    )
    (MLP_layer_2): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=496, out_features=496, bias=True)
        (Activation): ReLU()
      )
    )
    (MLP_layer_3): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=496, out_features=496, bias=True)
        (Activation): ReLU()
      )
    )
    (MLP_layer_4): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=496, out_features=496, bias=True)
        (Activation): ReLU()
      )
    )
    (MLP_layer_5): MLP(
      (block): Sequential(
        (Linear)

In [12]:
optimizer = torch.optim.Adam(ncf.parameters(), lr=0.004)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
criterion = torch.nn.BCELoss()
n_epochs = 7
device = "cpu"

In [13]:
%%time
train(
    ncf,
    train_loader,
    optimizer,
    scheduler,
    criterion,
    n_epochs,
    val_loader=val_loader,
    df_val=df_val,
    cos_dist=cos_dist,
    popularity=popularity,
    verbose=True,
)

Epochs:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 0
Train loss: 0.37612


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.5749),
 'NDCG @ 15': tensor(0.2777),
 'Diversity (ILD)': tensor(0.1419),
 'Novelty (EPC)': tensor(0.6912)}


...

{'Hit rate @ 15': tensor(0.7163),
 'NDCG @ 15': tensor(0.3679),
 'Diversity (ILD)': tensor(0.1142),
 'Novelty (EPC)': tensor(0.7641)}


  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 6
Train loss: 0.24041


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.7155),
 'NDCG @ 15': tensor(0.3718),
 'Diversity (ILD)': tensor(0.1096),
 'Novelty (EPC)': tensor(0.7733)}
CPU times: user 35min 18s, sys: 1h 27min 4s, total: 2h 2min 22s
Wall time: 13min 48s


In [14]:
df_test["pred"] = predict(ncf, test_loader, verbose=False).numpy()
pred, target, pred_items = split_test_df(df_test, "userId", "movieId", "pred", "action")
reccomendation_report(pred, target, pred_items, cos_dist, popularity, k=15)

{'Hit rate @ 15': tensor(0.7243),
 'NDCG @ 15': tensor(0.3734),
 'Diversity (ILD)': tensor(0.1094),
 'Novelty (EPC)': tensor(0.7721)}

## Test user recommendations

In [15]:
df_test_user = add_not_watched_movies(new_user_id, df_test, df_train, df_movies)
test_user_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test_user["userId"]), torch.tensor(df_test_user["movieId"])
    ),
    batch_size=4096,
    shuffle=False,
)
df_test_user["pred"] = predict(ncf, test_user_loader, verbose=False).numpy()
display(create_test_user_display_df(df_test_user, df_movies, "pred"))

,userId,movieId,name,genre,pred
0,6040,10,"American President, The (1995)",Comedy|Drama|Romance,0.962604
1,6040,1854,There's Something About Mary (1998),Comedy,0.934260
2,6040,523,Schindler's List (1993),Drama|War,0.908891
3,6040,1720,"Wedding Singer, The (1998)",Comedy|Romance,0.905256
4,6040,1656,Good Will Hunting (1997),Drama,0.890997
5,6040,1078,"Crying Game, The (1992)",Drama|Romance|War,0.882539
6,6040,2222,Edward Scissorhands (1990),Drama|Romance,0.879210
7,6040,1899,"Breakfast Club, The (1985)",Comedy|Drama,0.861292
8,6040,1878,West Side Story (1961),Musical|Romance,0.804010
9,6040,1455,Liar Liar (1997),Comedy,0.802889
